For two dataframes merge together, if merging columns on columns, the DataFrame indexes will be ignored. 

After the two dataframes merge, the index of the merged dataframe is reset. 

This could an ignored part in one large project and can cause problems in the later multiple dataframe combination or calculation.

Hence, I provide several examples to explain why it is important to be careful when do pd.merge, and what the safest way is when we do the merging.

In [71]:
# import libraries
import pandas as pd
import random

#### Data setup

In [10]:
account_number = random.sample(range(100000000, 800000000), 1000)

In [33]:
index_list = list(range(1000, 2000))

In [13]:
account_values = random.sample(range(1, 2000), 1000)

In [35]:
df_test_1 = pd.DataFrame(data={"account_number": account_number, "account_values": account_values}, index=index_list)

In [16]:
account_price = random.sample(range(1, 10000), 1000)

In [42]:
df_test_2 = pd.DataFrame(data={"account_number": account_number, "account_price": account_price}, index= index_list)

In [43]:
(df_test_1["account_number"] == df_test_2["account_number"]).all()

True

#### Case 1: merge two dataframes with the same indices, but not [0, 1, 2, 3, ...].

In [44]:
df_test_3 = df_test_1.merge(df_test_2, on=["account_number"])

In [47]:
df_test_1

,account_number,account_values
1000,108928675,791
1001,282411904,300
1002,299022651,1372
1003,508080513,750
1004,470639814,1932
...,...,...
1995,736240448,698
1996,374564362,1280
1997,426904865,1969
1998,587830205,205


In [46]:
df_test_3

,account_number,account_values,account_price
0,108928675,791,8244
1,282411904,300,1686
2,299022651,1372,7038
3,508080513,750,6346
4,470639814,1932,743
...,...,...,...
995,736240448,698,9876
996,374564362,1280,6227
997,426904865,1969,9051
998,587830205,205,6636


Even though df_test_1 and df_test_2 are with the same index, the df_test_3's index is reset from 0. When we compare the two series, it is not comparable, since the index are different between df_test_1 and df_test_3.

In [45]:
(df_test_1["account_number"] == df_test_3["account_number"]).all()

ValueError: Can only compare identically-labeled Series objects

However, when we compare the values of the two series, they are equal. Hence, the order of the accounts remain the same.

In [48]:
(df_test_1["account_number"].values == df_test_3["account_number"].values).all()

True

In [63]:
df_test_3_1 = df_test_1.merge(df_test_2, left_index=True, right_index=True)

In [64]:
df_test_3_1

,account_number_x,account_values,account_bin,account_number_y,account_price
1000,108928675,791,1,108928675,8244
1001,282411904,300,2,282411904,1686
1002,299022651,1372,3,299022651,7038
1003,508080513,750,4,508080513,6346
1004,470639814,1932,5,470639814,743
...,...,...,...,...,...
1995,736240448,698,1,736240448,9876
1996,374564362,1280,2,374564362,6227
1997,426904865,1969,3,426904865,9051
1998,587830205,205,4,587830205,6636


In [67]:
(df_test_1["account_number"].values == df_test_3_1["account_number_y"].values).all()

True

#### Case 2: merge two dataframes with the same key, but in different order

In [86]:
df_test_4 = df_test_2.sort_values(by="account_number").reset_index(drop=True)

In [87]:
df_test_4

,account_number,account_price
0,100300106,9490
1,100779966,3094
2,100860944,9791
3,101263323,1587
4,101440533,2917
...,...,...
995,797132945,3938
996,797339355,3979
997,797717673,3447
998,797925623,7960


In [88]:
df_test_4_1 = df_test_1.merge(df_test_4, on="account_number")

In [89]:
df_test_4_1

,account_number,account_values,account_bin,account_price
0,108928675,791,1,8244
1,282411904,300,2,1686
2,299022651,1372,3,7038
3,508080513,750,4,6346
4,470639814,1932,5,743
...,...,...,...,...
995,736240448,698,1,9876
996,374564362,1280,2,6227
997,426904865,1969,3,9051
998,587830205,205,4,6636


In [90]:
(df_test_1["account_number"].values == df_test_4_1["account_number"].values).all()

True

In [91]:
df_test_4_2 = df_test_1.merge(df_test_4, how="right", on="account_number")

In [92]:
(df_test_1["account_number"].values == df_test_4_2["account_number"].values).all()

False

In [93]:
(df_test_4["account_number"].values == df_test_4_2["account_number"].values).all()

True

### Case 3: merge two dataframes with different structure.

In [49]:
df_test_1['account_bin'] = [1, 2, 3, 4, 5]*200

In [72]:
# create another dataframe which has different indices. 
# This is to be used for merging on the column bin_value, and check if the account order changes.
bin_df = pd.DataFrame(data={"account_bin": [1,2 ,3 ,4, 5], "bin_value": [100, 105, 200, 150, 300]}, index=[0, 1, 2, 3, 4])

In [51]:
# pd.merge could do how="left", how="right", or how="inner. Its default setup is how="inner".
df_test_4 = df_test_1.merge(bin_df, on=["account_bin"])

In [75]:
(df_test_1.index == df_test_4.index).all()

False

In [54]:
(df_test_1["account_number"].values == df_test_4["account_number"].values).all()

False

Hence, the index is reset, and the order of the accounts **also changes**.

Join on **left**, to see how it changes.

In [55]:
df_test_5 = df_test_1.merge(bin_df, how="left", on=["account_bin"])

In [76]:
(df_test_1.index == df_test_5.index).all()

False

In [58]:
(df_test_1["account_number"].values == df_test_5["account_number"].values).all()

True

Hence, the index is reset, and the order of the accounts **doesn't** changes.

### The safest way to do pd.merge is to reset_index() before merge, and then set_index("index") if left_index is what you want.

In [69]:
df_test_6 = df_test_1.reset_index().merge(bin_df, how="left", on=["account_bin"]).set_index("index")

In [70]:
(df_test_1["account_number"] == df_test_6["account_number"]).all() # the merged df is kept in the same account order.

True

**Conclusions**:
    
1. pd.merge reset the index after merge.
    
2. pd.merge could merge the two dataframes with the same columns(account_number here, the merge key, which could be in different order)to generate the same order accounts in the merged dataframe as the left dataframe(how="left" or how="inner") or as the right dataframe(how="right"), even though reset index. 
    
3. When merge two different dataframes in terms of index and columns, the order of the merged dataframe is the same with the left dataframe if we define how="left". Otherwise, if we don't define how, it is taken as the default how="inner", and change the order of the rows.